In [6]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.orm import Session, sessionmaker, scoped_session
from chromedriver_py import binary_path
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from sqlalchemy.util.compat import contextmanager
from selenium.common.exceptions import NoSuchElementException
import random

In [8]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
opts = Options()
# opts.headless = True
#driver = webdriver.Chrome(options=opts, executable_path=binary_path)

In [9]:
opts = Options()
# opts.headless = True
driver = webdriver.Chrome(ChromeDriverManager().install(), options=opts)#, executable_path=binary_path)



additional_cases_to_scrape = []

x_case_number = []
x_case_status = []
x_judge_name = []
x_next_event = []
x_arrested_date = []
x_arresting_agency = []
x_arresting_agency_report = []
x_court_of_appeals_case = []
x_defendant_number = []
x_defendant_name = []
x_defendant_status = []
x_defendant_dob = []
x_defendant_race = []
x_defendant_sex = []
x_other_cases = []
x_co_defendants = []
charge_type = []
charge_statute = []
charge_desc = []
charge_disposition = []
bond_number = []
bond_amount = []
bond_type = []
bond_date_set = []
bond_date_posted = []
bond_bond_company = []
event_date = []
event_description = []
charge_dfs = []
actions_dfs = []
bonds_dfs = []


[WDM] - Downloading: 100%|██████████| 8.04M/8.04M [00:01<00:00, 4.68MB/s]
/var/folders/rb/4k3rm5dn3_5gbmbfgfhgdy280000gn/T/ipykernel_54228/2906633962.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=opts)#, executable_path=binary_path)


In [10]:
def search_case_number(case_number):
    """upon landing on the search page, this function will navigate the agreement and then enter a case 
    number to search criminal records and click the submit button.
    Be sure to already be on the case search page when you run this function."""
    WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//*[(@id = 'SheetContentPlaceHolder_rbCrCase')]"))).click()
    driver.implicitly_wait(2)
    
    if len(driver.find_elements(By.ID, "SheetContentPlaceHolder_btnYes")) > 0:
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "SheetContentPlaceHolder_btnYes"))).click()
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//*[(@id = 'SheetContentPlaceHolder_rbCrCase')]"))).click()
        driver.implicitly_wait(3)

    text_box = driver.find_element(By.XPATH, "//input[(@type = 'text')]")
    text_box.clear()
    text_box.send_keys(case_number)
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "SheetContentPlaceHolder_criminalCaseSearch_btnSubmitCase"))).click()
    driver.implicitly_wait(random.randint(4,15))
    return driver

def get_table(case_number: str, header_xpath: str, data_xpath: str) -> pd.DataFrame:
    headers = driver.find_elements(By.XPATH, header_xpath)
    data = driver.find_elements(By.XPATH, data_xpath)
    data_list = [d.text for d in data]
    table = {}
    for i, h in enumerate(headers):
        table[h.text.upper()]=data_list[i::len(headers)]
    df = pd.DataFrame(table)
    df["CASE_NUMBER"] = case_number
    return df

In [11]:
driver.get("https://cpdocket.cp.cuyahogacounty.us/Search.aspx") 

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
referer="https://cpdocket.cp.cuyahogacounty.us/CR_CaseInformation_Docket.aspx?q=JGza6PovFtECU5ADsehJFA2"

In [ ]:
//*[@id="SheetContentPlaceHolder_caseDocket_gvDocketInformation"]/tbody/tr[3]/td[6]/a

In [ ]:
<a title="View Docket Image" href="DisplayImageList.aspx?q=b25wMZvfMXXbV3SlcWWeZlBYBEHPb5xb0" target="_blank"
<img title="View Docket Image" src="images/ImageSheet.png" alt="View Docket Image"></a>

In [4]:
r_obj = requests.Session()
url = "https://cpdocket.cp.cuyahogacounty.us/CR_CaseInformation_Docket.aspx?q=JGza6PovFtECU5ADsehJFA2" # i.e. http://bisegrw.edu.pk/
r_soup = r_obj.get(url)
soup = BeautifulSoup(r_soup.content , "lxml")


In [6]:
hidden_inputs = soup.find_all("input",type="hidden")

In [23]:
soup.find_all("submitButton")

[]

In [24]:
data = {
hidden_inputs[0]['name']:hidden_inputs[0]['value'],
hidden_inputs[1]['name']:hidden_inputs[1]['value'],
hidden_inputs[2]['name']:hidden_inputs[2]['value'],
'ctl00$SheetContentPlaceHolder$btnYes':'Yes'}

url_needed = "aspx_endpoint" # i.e. http://bisegrw.edu.pk/result-card-matric.html
final = r_obj.post(url,verify=False,data=data)
soup1 = BeautifulSoup(final.content,"lxml")
detail_tab = soup1.find_all("table")

In [25]:
detail_tab.pr   

[<table style="height:100px; width:100%; padding:0px; border-spacing:0px; ">
 <tr>
 <td style="width:5%"></td>
 <td style="width:95%; overflow:hidden;">
 <h1 class="art-logo-name" id="name-text">
                     Cuyahoga County
                     <br/>
                     Clerk of Courts
                 </h1>
 </td>
 </tr>
 </table>,
 <table style="width:100%; height:100%; padding:0px; border-spacing:7px; ">
 <tr>
 <td style="text-align:right;">
 </td>
 <td style="text-align:right;">
 </td>
 </tr>
 </table>,
 <table style="background-color: #DCDCDC; width: 100%;">
 <tr>
 <td style="text-align:center; padding:20px;">
 <br/>
 <div id="SheetContentPlaceHolder_yesDiv">
 <p>
                                                 Welcome to the Case Records Search System of the Cuyahoga County Clerk of Courts. 
                                                 The services and/or information we provided through this Website are subject to these 
                                            

In [ ]:


r_obj = requests.Session()
url = "https://cpdocket.cp.cuyahogacounty.us/Search.aspx" # i.e. http://bisegrw.edu.pk/
r_soup = r_obj.get(url)
soup = BeautifulSoup(r_soup.content , "lxml")

hidden_inputs = soup.find_all("input",type="hidden")

data = {
hidden_inputs[0]['name']:hidden_inputs[0]['value'],
hidden_inputs[1]['name']:hidden_inputs[1]['value'],
hidden_inputs[2]['name']:hidden_inputs[2]['value'],
'other_input':'value',  # i.e. 'rno':'487431',
'submitButton':'submit'}

url_needed = "aspx_endpoint" # i.e. http://bisegrw.edu.pk/result-card-matric.html
final = r_obj.post(url_needed,verify=False,data=data)
soup1 = BeautifulSoup(final.content,"lxml")
detail_tab = soup1.find_all("table")



In [12]:
case_number="492405"

In [13]:
driver = search_case_number(case_number)

In [14]:
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.LINK_TEXT, "CR-07-492405-B"))).click()

In [34]:
first_table = driver.find_element(By.XPATH,"//table[1]")

In [36]:
li = first_table.find_elements(By.TAG_NAME, "td")

In [41]:
li[2].text

IndexError: list index out of range

In [23]:
hi = first_table.find_elements(By.TAG_NAME, "td")

In [24]:
hi[0].text

'Number:'

In [25]:
hi[1].text

'166616'